# MLOps

```{admonition} Interactive Learning
:class: tip
🚀 **Launch Binder**: Click the rocket icon above to run this chapter interactively!

💾 **Save Your Work**: Download your completed notebook when done.

🏗️ **Build Locally**: Ready for serious development? [Fork the repo](https://github.com/your-org/tinytorch) and work locally with the full `tito` workflow.
```



In [ ]:
#| default_exp core.mlops

#| export
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import time
import json
from typing import Dict, List, Tuple, Optional, Any, Callable
from dataclasses import dataclass, field
from datetime import datetime, timedelta
from collections import defaultdict

# Import our dependencies - try from package first, then local modules
try:
    from tinytorch.core.tensor import Tensor
    from tinytorch.core.training import Trainer, MeanSquaredError, CrossEntropyLoss, Accuracy
    from tinytorch.core.benchmarking import TinyTorchPerf, StatisticalValidator
    from tinytorch.core.compression import quantize_layer_weights, prune_weights_by_magnitude
    from tinytorch.core.networks import Sequential
    from tinytorch.core.layers import Dense
    from tinytorch.core.activations import ReLU, Sigmoid, Softmax
except ImportError:
    # For development, import from local modules
    sys.path.append(os.path.join(os.path.dirname(__file__), '..', '01_tensor'))
    sys.path.append(os.path.join(os.path.dirname(__file__), '..', '09_training'))
    sys.path.append(os.path.join(os.path.dirname(__file__), '..', '12_benchmarking'))
    sys.path.append(os.path.join(os.path.dirname(__file__), '..', '10_compression'))
    sys.path.append(os.path.join(os.path.dirname(__file__), '..', '04_networks'))
    sys.path.append(os.path.join(os.path.dirname(__file__), '..', '03_layers'))
    sys.path.append(os.path.join(os.path.dirname(__file__), '..', '02_activations'))
    try:
        from tensor_dev import Tensor
        from training_dev import Trainer, MeanSquaredError, CrossEntropyLoss, Accuracy
        from benchmarking_dev import TinyTorchPerf, StatisticalValidator
        from compression_dev import quantize_layer_weights, prune_weights_by_magnitude
        from networks_dev import Sequential
        from layers_dev import Dense
        from activations_dev import ReLU, Sigmoid, Softmax
    except ImportError:
        print("⚠️  Development imports failed - some functionality may be limited")

In [ ]:
#| hide
#| export
def _should_show_plots():
    """Check if we should show plots (disable during testing)"""
    # Check multiple conditions that indicate we're in test mode
    is_pytest = (
        'pytest' in sys.modules or
        'test' in sys.argv or
        os.environ.get('PYTEST_CURRENT_TEST') is not None or
        any('test' in arg for arg in sys.argv) or
        any('pytest' in arg for arg in sys.argv)
    )
    
    # Show plots in development mode (when not in test mode)
    return not is_pytest

In [ ]:
print("🚀 TinyTorch MLOps Module")
print(f"NumPy version: {np.__version__}")
print(f"Python version: {sys.version_info.major}.{sys.version_info.minor}")
print("Ready to build production ML systems!")

## 📦 Where This Code Lives in the Final Package

**Learning Side:** You work in `modules/source/13_mlops/mlops_dev.py`  
**Building Side:** Code exports to `tinytorch.core.mlops`

```python
# Final package structure:
from tinytorch.core.mlops import ModelMonitor, DriftDetector, MLOpsPipeline
from tinytorch.core.training import Trainer  # Reuse your training system
from tinytorch.core.benchmarking import TinyTorchPerf  # Reuse your benchmarking
from tinytorch.core.compression import quantize_layer_weights  # Reuse compression
```

**Why this matters:**
- **Integration:** MLOps orchestrates all TinyTorch components
- **Reusability:** Uses everything you've built in previous modules
- **Production:** Real-world ML system lifecycle management
- **Maintainability:** Systems that keep working over time

## What is MLOps?

### The Production Reality: Models Degrade Over Time
You've built an amazing ML system:
- **Training pipeline**: Produces high-quality models
- **Compression**: Optimizes models for deployment
- **Kernels**: Accelerates inference
- **Benchmarking**: Measures performance

But there's a critical problem: **Models degrade over time without maintenance.**

### Why Models Fail in Production
1. **Data drift**: Input data distribution changes
2. **Concept drift**: Relationship between inputs and outputs changes
3. **Performance degradation**: Accuracy drops over time
4. **System changes**: Infrastructure updates break assumptions

### The MLOps Solution
**MLOps** (Machine Learning Operations) is the practice of maintaining ML systems in production:
- **Monitor**: Track model performance continuously
- **Detect**: Identify when models are failing
- **Respond**: Automatically retrain and redeploy
- **Validate**: Ensure new models are actually better

### Real-World Examples
- **Netflix**: Recommendation models retrain when viewing patterns change
- **Uber**: Demand prediction models adapt to new cities and events
- **Google**: Search ranking models update as web content evolves
- **Tesla**: Autonomous driving models improve with new driving data

### The Complete TinyTorch Lifecycle
```
Data → Training → Compression → Kernels → Benchmarking → Monitor → Detect → Retrain → Deploy
                                                             ↑__________________________|
```

MLOps closes this loop, creating **self-maintaining systems**.

## Step 1: Performance Drift Monitor - Tracking Model Health

### The Problem: Silent Model Degradation
Without monitoring, you won't know when your model stops working:
- **Accuracy drops** from 95% to 85% over 3 months
- **Latency increases** as data patterns change
- **System failures** go unnoticed until user complaints

### The Solution: Continuous Performance Monitoring
Track key metrics over time:
- **Accuracy/Error rates**: Primary model performance
- **Latency/Throughput**: System performance
- **Data statistics**: Input distribution changes
- **System health**: Infrastructure metrics

### What We'll Build
A `ModelMonitor` that:
1. **Tracks performance** over time
2. **Stores metric history** for trend analysis
3. **Detects degradation** when metrics drop
4. **Alerts** when thresholds are crossed

### Real-World Applications
- **E-commerce**: Monitor recommendation click-through rates
- **Finance**: Track fraud detection false positive rates
- **Healthcare**: Monitor diagnostic accuracy over time
- **Autonomous vehicles**: Track object detection confidence scores

In [ ]:
#| export
@dataclass
class ModelMonitor:
    """
    Monitors ML model performance over time and detects degradation.
    
    Tracks key metrics, stores history, and alerts when performance drops.
    """
    
    def __init__(self, model_name: str, baseline_accuracy: float = 0.95):
        """
        TODO: Initialize the ModelMonitor for tracking model performance.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Store the model_name and baseline_accuracy
        2. Create empty lists to store metric history:
           - accuracy_history: List[float] 
           - latency_history: List[float]
           - timestamp_history: List[datetime]
        3. Set performance thresholds:
           - accuracy_threshold: baseline_accuracy * 0.9 (10% drop triggers alert)
           - latency_threshold: 200.0 (milliseconds)
        4. Initialize alert flags:
           - accuracy_alert: False
           - latency_alert: False
        
        EXAMPLE USAGE:
        ```python
        monitor = ModelMonitor("image_classifier", baseline_accuracy=0.93)
        monitor.record_performance(accuracy=0.92, latency=150.0)
        alerts = monitor.check_alerts()
        ```
        
        IMPLEMENTATION HINTS:
        - Use self.model_name = model_name
        - Initialize lists with self.accuracy_history = []
        - Use datetime.now() for timestamps
        - Set thresholds relative to baseline (e.g., 90% of baseline)
        
        LEARNING CONNECTIONS:
        - This builds on benchmarking concepts from Module 12
        - Performance tracking is essential for production systems
        - Thresholds prevent false alarms while catching real issues
        """
        ### BEGIN SOLUTION
    # YOUR CODE HERE
    raise NotImplementedError()
        ### END SOLUTION
    
    def record_performance(self, accuracy: float, latency: float):
        """
        TODO: Record a new performance measurement.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Get current timestamp with datetime.now()
        2. Append accuracy to self.accuracy_history
        3. Append latency to self.latency_history
        4. Append timestamp to self.timestamp_history
        5. Check if accuracy is below threshold:
           - If accuracy < self.accuracy_threshold: set self.accuracy_alert = True
           - Else: set self.accuracy_alert = False
        6. Check if latency is above threshold:
           - If latency > self.latency_threshold: set self.latency_alert = True
           - Else: set self.latency_alert = False
        
        EXAMPLE BEHAVIOR:
        ```python
        monitor.record_performance(0.94, 120.0)  # Good performance
        monitor.record_performance(0.84, 250.0)  # Triggers both alerts
        ```
        
        IMPLEMENTATION HINTS:
        - Use datetime.now() for timestamps
        - Update alert flags based on current measurement
        - Don't forget to store all three values (accuracy, latency, timestamp)
        """
        ### BEGIN SOLUTION
    # YOUR CODE HERE
    raise NotImplementedError()
        ### END SOLUTION
    
    def check_alerts(self) -> Dict[str, Any]:
        """
        TODO: Check current alert status and return alert information.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Create result dictionary with basic info:
           - "model_name": self.model_name
           - "accuracy_alert": self.accuracy_alert
           - "latency_alert": self.latency_alert
        2. If accuracy_alert is True, add:
           - "accuracy_message": f"Accuracy below threshold: {current_accuracy:.3f} < {self.accuracy_threshold:.3f}"
           - "current_accuracy": most recent accuracy from history
        3. If latency_alert is True, add:
           - "latency_message": f"Latency above threshold: {current_latency:.1f}ms > {self.latency_threshold:.1f}ms"
           - "current_latency": most recent latency from history
        4. Add overall alert status:
           - "any_alerts": True if any alert is active
        
        EXAMPLE RETURN:
        ```python
        {
            "model_name": "image_classifier",
            "accuracy_alert": True,
            "latency_alert": False,
            "accuracy_message": "Accuracy below threshold: 0.840 < 0.855",
            "current_accuracy": 0.840,
            "any_alerts": True
        }
        ```
        
        IMPLEMENTATION HINTS:
        - Use self.accuracy_history[-1] for most recent values
        - Format numbers with f-strings for readability
        - Include both alert flags and descriptive messages
        """
        ### BEGIN SOLUTION
    # YOUR CODE HERE
    raise NotImplementedError()
        ### END SOLUTION
    
    def get_performance_trend(self) -> Dict[str, Any]:
        """
        TODO: Analyze performance trends over time.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Check if we have enough data (at least 2 measurements)
        2. Calculate accuracy trend:
           - If accuracy_history has < 2 points: trend = "insufficient_data"
           - Else: compare recent avg (last 3) vs older avg (first 3)
           - If recent > older: trend = "improving"
           - If recent < older: trend = "degrading"
           - Else: trend = "stable"
        3. Calculate similar trend for latency
        4. Return dictionary with:
           - "measurements_count": len(self.accuracy_history)
           - "accuracy_trend": trend analysis
           - "latency_trend": trend analysis
           - "baseline_accuracy": self.baseline_accuracy
           - "current_accuracy": most recent accuracy (if available)
        
        EXAMPLE RETURN:
        ```python
        {
            "measurements_count": 10,
            "accuracy_trend": "degrading",
            "latency_trend": "stable",
            "baseline_accuracy": 0.95,
            "current_accuracy": 0.87
        }
        ```
        
        IMPLEMENTATION HINTS:
        - Use len(self.accuracy_history) for data count
        - Use np.mean() for calculating averages
        - Handle edge cases (empty history, insufficient data)
        """
        ### BEGIN SOLUTION
    # YOUR CODE HERE
    raise NotImplementedError()
        ### END SOLUTION

### 🧪 Test Your Performance Monitor

Once you implement the `ModelMonitor` class above, run this cell to test it:

In [ ]:
def test_model_monitor():
    """Test ModelMonitor implementation"""
    print("🔬 Unit Test: Performance Drift Monitor...")
    
    # Test initialization
    monitor = ModelMonitor("test_model", baseline_accuracy=0.90)
    
    assert monitor.model_name == "test_model"
    assert monitor.baseline_accuracy == 0.90
    assert monitor.accuracy_threshold == 0.81  # 90% of 0.90
    assert monitor.latency_threshold == 200.0
    assert not monitor.accuracy_alert
    assert not monitor.latency_alert
    
    # Test good performance (no alerts)
    monitor.record_performance(accuracy=0.92, latency=150.0)
    
    alerts = monitor.check_alerts()
    assert not alerts["accuracy_alert"]
    assert not alerts["latency_alert"]
    assert not alerts["any_alerts"]
    
    # Test accuracy degradation
    monitor.record_performance(accuracy=0.80, latency=150.0)  # Below threshold
    
    alerts = monitor.check_alerts()
    assert alerts["accuracy_alert"]
    assert not alerts["latency_alert"]
    assert alerts["any_alerts"]
    assert "Accuracy below threshold" in alerts["accuracy_message"]
    
    # Test latency degradation
    monitor.record_performance(accuracy=0.85, latency=250.0)  # Above threshold
    
    alerts = monitor.check_alerts()
    assert not alerts["accuracy_alert"]  # Back above threshold
    assert alerts["latency_alert"]
    assert alerts["any_alerts"]
    assert "Latency above threshold" in alerts["latency_message"]
    
    # Test trend analysis
    # Add more measurements to test trends
    for i in range(5):
        monitor.record_performance(accuracy=0.90 - i*0.02, latency=120.0 + i*10)
    
    trend = monitor.get_performance_trend()
    assert trend["measurements_count"] >= 5
    assert trend["accuracy_trend"] in ["improving", "degrading", "stable"]
    assert trend["latency_trend"] in ["improving", "degrading", "stable"]
    assert trend["baseline_accuracy"] == 0.90
    
    print("✅ ModelMonitor initialization works correctly")
    print("✅ Performance recording and alert detection work")
    print("✅ Alert checking returns proper format")
    print("✅ Trend analysis provides meaningful insights")
    print("📈 Progress: Performance Drift Monitor ✓")

# Run the test
test_model_monitor()

## Step 2: Simple Drift Detection - Detecting Data Changes

### The Problem: Silent Data Distribution Changes
Your model was trained on specific data patterns, but production data evolves:
- **Seasonal changes**: E-commerce traffic patterns change during holidays
- **User behavior shifts**: App usage patterns evolve over time
- **External factors**: Economic conditions affect financial predictions
- **System changes**: New data sources introduce different distributions

### The Solution: Statistical Drift Detection
Compare current data to baseline data using statistical tests:
- **Kolmogorov-Smirnov test**: Detects distribution changes
- **Mean/Standard deviation shifts**: Simple but effective
- **Population stability index**: Common in industry
- **Chi-square test**: For categorical features

### What We'll Build
A `DriftDetector` that:
1. **Stores baseline data** from training time
2. **Compares new data** to baseline using statistical tests
3. **Detects significant changes** in distribution
4. **Provides interpretable results** for debugging

### Real-World Applications
- **Fraud detection**: New fraud patterns emerge constantly
- **Recommendation systems**: User preferences shift over time
- **Medical diagnosis**: Patient demographics change
- **Computer vision**: Camera quality, lighting conditions evolve

In [ ]:
#| export
class DriftDetector:
    """
    Detects data drift by comparing current data distributions to baseline.
    
    Uses statistical tests to identify significant changes in data patterns.
    """
    
    def __init__(self, baseline_data: np.ndarray, feature_names: Optional[List[str]] = None):
        """
        TODO: Initialize the DriftDetector with baseline data.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Store baseline_data and feature_names
        2. Calculate baseline statistics:
           - baseline_mean: np.mean(baseline_data, axis=0)
           - baseline_std: np.std(baseline_data, axis=0)
           - baseline_min: np.min(baseline_data, axis=0)
           - baseline_max: np.max(baseline_data, axis=0)
        3. Set drift detection threshold (default: 0.05 for 95% confidence)
        4. Initialize drift history storage:
           - drift_history: List[Dict] to store drift test results
        
        EXAMPLE USAGE:
        ```python
        baseline = np.random.normal(0, 1, (1000, 3))
        detector = DriftDetector(baseline, ["feature1", "feature2", "feature3"])
        drift_result = detector.detect_drift(new_data)
        ```
        
        IMPLEMENTATION HINTS:
        - Use axis=0 for column-wise statistics
        - Handle case when feature_names is None
        - Store original baseline_data for KS test
        - Set significance level (alpha) to 0.05
        """
        ### BEGIN SOLUTION
    # YOUR CODE HERE
    raise NotImplementedError()
        ### END SOLUTION
    
    def detect_drift(self, new_data: np.ndarray) -> Dict[str, Any]:
        """
        TODO: Detect drift by comparing new data to baseline.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Calculate new data statistics:
           - new_mean, new_std, new_min, new_max (same as baseline)
        2. Perform statistical tests for each feature:
           - KS test: from scipy.stats import ks_2samp (if available)
           - Mean shift test: |new_mean - baseline_mean| / baseline_std > 2
           - Std shift test: |new_std - baseline_std| / baseline_std > 0.5
        3. Create result dictionary:
           - "drift_detected": True if any feature shows drift
           - "feature_drift": Dict with per-feature results
           - "summary": Overall drift description
        4. Store result in drift_history
        
        EXAMPLE RETURN:
        ```python
        {
            "drift_detected": True,
            "feature_drift": {
                "feature1": {"mean_drift": True, "std_drift": False, "ks_pvalue": 0.001},
                "feature2": {"mean_drift": False, "std_drift": True, "ks_pvalue": 0.3}
            },
            "summary": "Drift detected in 2/3 features"
        }
        ```
        
        IMPLEMENTATION HINTS:
        - Use try-except for KS test (may not be available)
        - Check each feature individually
        - Use absolute values for difference checks
        - Count how many features show drift
        """
        ### BEGIN SOLUTION
    # YOUR CODE HERE
    raise NotImplementedError()
        ### END SOLUTION
    
    def get_drift_history(self) -> List[Dict]:
        """
        TODO: Return the complete drift detection history.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Return self.drift_history
        2. Include timestamp and result for each detection
        3. Format for easy analysis
        
        EXAMPLE RETURN:
        ```python
        [
            {
                "timestamp": datetime(2024, 1, 1, 12, 0),
                "result": {"drift_detected": False, "drift_count": 0, ...}
            },
            {
                "timestamp": datetime(2024, 1, 2, 12, 0),
                "result": {"drift_detected": True, "drift_count": 2, ...}
            }
        ]
        ```
        """
        ### BEGIN SOLUTION
    # YOUR CODE HERE
    raise NotImplementedError()
        ### END SOLUTION

### 🧪 Test Your Drift Detector

Once you implement the `DriftDetector` class above, run this cell to test it:

In [ ]:
def test_drift_detector():
    """Test DriftDetector implementation"""
    print("🔬 Unit Test: Simple Drift Detection...")
    
    # Create baseline data
    np.random.seed(42)
    baseline_data = np.random.normal(0, 1, (1000, 3))
    feature_names = ["feature1", "feature2", "feature3"]
    
    detector = DriftDetector(baseline_data, feature_names)
    
    # Test initialization
    assert detector.baseline_data.shape == (1000, 3)
    assert len(detector.feature_names) == 3
    assert detector.feature_names == feature_names
    assert detector.significance_level == 0.05
    
    # Test no drift (similar data)
    no_drift_data = np.random.normal(0, 1, (500, 3))
    result = detector.detect_drift(no_drift_data)
    
    assert "drift_detected" in result
    assert "feature_drift" in result
    assert "summary" in result
    assert len(result["feature_drift"]) == 3
    
    # Test clear drift (shifted data)
    drift_data = np.random.normal(3, 1, (500, 3))  # Mean shifted by 3
    result = detector.detect_drift(drift_data)
    
    assert result["drift_detected"] == True
    assert result["drift_count"] > 0
    assert "Drift detected" in result["summary"]
    
    # Check feature-level drift detection
    for feature_name in feature_names:
        feature_result = result["feature_drift"][feature_name]
        assert "mean_drift" in feature_result
        assert "std_drift" in feature_result
        assert "mean_change" in feature_result
    
    # Test drift history
    history = detector.get_drift_history()
    assert len(history) >= 2  # At least 2 drift checks
    assert all("timestamp" in entry for entry in history)
    assert all("result" in entry for entry in history)
    
    print("✅ DriftDetector initialization works correctly")
    print("✅ No-drift detection works (similar data)")
    print("✅ Clear drift detection works (shifted data)")
    print("✅ Feature-level drift analysis works")
    print("✅ Drift history tracking works")
    print("📈 Progress: Simple Drift Detection ✓")

# Run the test
test_drift_detector()

## Step 3: Retraining Trigger System - Automated Response to Issues

### The Problem: Manual Intervention Required
You can detect when models are failing, but someone needs to:
- **Notice the alerts** (requires constant monitoring)
- **Decide to retrain** (requires domain expertise)
- **Execute retraining** (requires technical knowledge)
- **Validate results** (requires ML expertise)

### The Solution: Automated Retraining Pipeline
Create a system that automatically responds to performance degradation:
- **Threshold-based triggers**: Automatically start retraining when performance drops
- **Reuse existing components**: Use your training pipeline from Module 09
- **Intelligent scheduling**: Avoid unnecessary retraining
- **Validation before deployment**: Ensure new models are actually better

### What We'll Build
A `RetrainingTrigger` that:
1. **Monitors model performance** using ModelMonitor
2. **Detects drift** using DriftDetector
3. **Triggers retraining** when conditions are met
4. **Orchestrates the process** using existing TinyTorch components

### Real-World Applications
- **A/B testing platforms**: Automatically update models based on performance
- **Recommendation engines**: Retrain when user behavior changes
- **Fraud detection**: Adapt to new fraud patterns automatically
- **Predictive maintenance**: Update models as equipment ages

In [ ]:
#| export
class RetrainingTrigger:
    """
    Automated retraining system that responds to model performance degradation.
    
    Orchestrates the complete retraining workflow using existing TinyTorch components.
    """
    
    def __init__(self, model, training_data, validation_data, trainer_class=None):
        """
        TODO: Initialize the RetrainingTrigger system.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Store the model, training_data, and validation_data
        2. Set up the trainer_class (use provided or default to simple trainer)
        3. Initialize trigger conditions:
           - accuracy_threshold: 0.85 (trigger retraining if accuracy < 85%)
           - drift_threshold: 2 (trigger if drift detected in 2+ features)
           - min_time_between_retrains: 24 hours (avoid too frequent retraining)
        4. Initialize tracking variables:
           - last_retrain_time: datetime.now()
           - retrain_history: List[Dict] to store retraining results
        
        EXAMPLE USAGE:
        ```python
        trigger = RetrainingTrigger(model, train_data, val_data)
        should_retrain = trigger.check_trigger_conditions(monitor, drift_detector)
        if should_retrain:
            new_model = trigger.execute_retraining()
        ```
        
        IMPLEMENTATION HINTS:
        - Store references to data for retraining
        - Set reasonable default thresholds
        - Use datetime for time tracking
        - Initialize empty history list
        """
        ### BEGIN SOLUTION
    # YOUR CODE HERE
    raise NotImplementedError()
        ### END SOLUTION
    
    def check_trigger_conditions(self, monitor: ModelMonitor, drift_detector: DriftDetector) -> Dict[str, Any]:
        """
        TODO: Check if retraining should be triggered.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Get current time and check time since last retrain:
           - time_since_last = (current_time - self.last_retrain_time).total_seconds()
           - too_soon = time_since_last < self.min_time_between_retrains
        2. Check monitor alerts:
           - Get alerts from monitor.check_alerts()
           - accuracy_trigger = alerts["accuracy_alert"]
        3. Check drift status:
           - Get latest drift from drift_detector.drift_history
           - drift_trigger = drift_count >= self.drift_threshold
        4. Determine overall trigger status:
           - should_retrain = (accuracy_trigger or drift_trigger) and not too_soon
        5. Return comprehensive result dictionary
        
        EXAMPLE RETURN:
        ```python
        {
            "should_retrain": True,
            "accuracy_trigger": True,
            "drift_trigger": False,
            "time_trigger": True,
            "reasons": ["Accuracy below threshold: 0.82 < 0.85"],
            "time_since_last_retrain": 86400
        }
        ```
        
        IMPLEMENTATION HINTS:
        - Use .total_seconds() for time differences
        - Collect all trigger reasons in a list
        - Handle empty drift history gracefully
        - Provide detailed feedback for debugging
        """
        ### BEGIN SOLUTION
    # YOUR CODE HERE
    raise NotImplementedError()
        ### END SOLUTION
    
    def execute_retraining(self) -> Dict[str, Any]:
        """
        TODO: Execute the retraining process.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Record start time and create result dictionary
        2. Simulate training process:
           - Create simple model (copy of original architecture)
           - Simulate training with random improvement
           - Calculate new performance (baseline + random improvement)
        3. Validate new model:
           - Compare old vs new performance
           - Only deploy if new model is better
        4. Update tracking:
           - Update last_retrain_time
           - Add entry to retrain_history
        5. Return comprehensive result
        
        EXAMPLE RETURN:
        ```python
        {
            "success": True,
            "old_accuracy": 0.82,
            "new_accuracy": 0.91,
            "improvement": 0.09,
            "deployed": True,
            "training_time": 45.2,
            "timestamp": datetime(2024, 1, 1, 12, 0)
        }
        ```
        
        IMPLEMENTATION HINTS:
        - Use time.time() for timing
        - Simulate realistic training time (random 30-60 seconds)
        - Add random improvement (0.02-0.08 accuracy boost)
        - Only deploy if new model is better
        - Store detailed results for analysis
        """
        ### BEGIN SOLUTION
    # YOUR CODE HERE
    raise NotImplementedError()
        ### END SOLUTION
    
    def get_retraining_history(self) -> List[Dict]:
        """
        TODO: Return the complete retraining history.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Return self.retrain_history
        2. Include all retraining attempts with results
        
        EXAMPLE RETURN:
        ```python
        [
            {
                "success": True,
                "old_accuracy": 0.82,
                "new_accuracy": 0.89,
                "improvement": 0.07,
                "deployed": True,
                "training_time": 42.1,
                "timestamp": datetime(2024, 1, 1, 12, 0)
            }
        ]
        ```
        """
        ### BEGIN SOLUTION
    # YOUR CODE HERE
    raise NotImplementedError()
        ### END SOLUTION

### 🧪 Test Your Retraining Trigger

Once you implement the `RetrainingTrigger` class above, run this cell to test it:

In [ ]:
def test_retraining_trigger():
    """Test RetrainingTrigger implementation"""
    print("🔬 Unit Test: Retraining Trigger System...")
    
    # Create mock model and data
    model = "mock_model"
    train_data = np.random.normal(0, 1, (1000, 10))
    val_data = np.random.normal(0, 1, (200, 10))
    
    # Create retraining trigger
    trigger = RetrainingTrigger(model, train_data, val_data)
    
    # Test initialization
    assert trigger.model == model
    assert trigger.accuracy_threshold == 0.82
    assert trigger.drift_threshold == 1
    assert trigger.min_time_between_retrains == 24 * 60 * 60
    
    # Create monitor and drift detector for testing
    monitor = ModelMonitor("test_model", baseline_accuracy=0.90)
    baseline_data = np.random.normal(0, 1, (1000, 3))
    drift_detector = DriftDetector(baseline_data)
    
    # Test no trigger conditions (good performance)
    monitor.record_performance(accuracy=0.92, latency=150.0)
    no_drift_data = np.random.normal(0, 1, (500, 3))
    drift_detector.detect_drift(no_drift_data)
    
    conditions = trigger.check_trigger_conditions(monitor, drift_detector)
    assert not conditions["should_retrain"]
    assert not conditions["accuracy_trigger"]
    assert not conditions["drift_trigger"]
    
    # Test accuracy trigger
    monitor.record_performance(accuracy=0.80, latency=150.0)  # Below threshold
    conditions = trigger.check_trigger_conditions(monitor, drift_detector)
    assert conditions["accuracy_trigger"]
    
    # Test drift trigger
    drift_data = np.random.normal(3, 1, (500, 3))  # Shifted data
    drift_detector.detect_drift(drift_data)
    conditions = trigger.check_trigger_conditions(monitor, drift_detector)
    assert conditions["drift_trigger"]
    
    # Test retraining execution
    result = trigger.execute_retraining()
    assert result["success"] == True
    assert "old_accuracy" in result
    assert "new_accuracy" in result
    assert "improvement" in result
    assert "deployed" in result
    assert "training_time" in result
    assert "timestamp" in result
    
    # Test retraining history
    history = trigger.get_retraining_history()
    assert len(history) >= 1
    assert all("timestamp" in entry for entry in history)
    assert all("success" in entry for entry in history)
    
    print("✅ RetrainingTrigger initialization works correctly")
    print("✅ Trigger condition checking works")
    print("✅ Accuracy and drift triggers work")
    print("✅ Retraining execution works")
    print("✅ Retraining history tracking works")
    print("📈 Progress: Retraining Trigger System ✓")

# Run the test
test_retraining_trigger()

## Step 4: Complete MLOps Pipeline - Integration and Deployment

### The Problem: Disconnected Components
You have built individual MLOps components, but they need to work together:
- **ModelMonitor**: Tracks performance over time
- **DriftDetector**: Identifies data distribution changes
- **RetrainingTrigger**: Automates retraining decisions
- **Need**: Integration layer that orchestrates everything

### The Solution: Complete MLOps Pipeline
Create a unified system that brings everything together:
- **Unified interface**: Single entry point for all MLOps operations
- **Automated workflows**: End-to-end automation from monitoring to deployment
- **Integration with TinyTorch**: Uses all previous modules seamlessly
- **Production-ready**: Handles edge cases and error conditions

### What We'll Build
An `MLOpsPipeline` that:
1. **Integrates all components** into a cohesive system
2. **Orchestrates the complete workflow** from monitoring to deployment
3. **Provides simple API** for production use
4. **Demonstrates the full TinyTorch ecosystem** working together

### Real-World Applications
- **End-to-end ML platforms**: MLflow, Kubeflow, SageMaker
- **Production ML systems**: Netflix, Uber, Google's ML infrastructure
- **Automated ML pipelines**: Continuous learning and deployment
- **ML monitoring platforms**: Datadog, New Relic for ML systems

In [ ]:
#| export
class MLOpsPipeline:
    """
    Complete MLOps pipeline that integrates all components.
    
    Orchestrates the full ML system lifecycle from monitoring to deployment.
    """
    
    def __init__(self, model, training_data, validation_data, baseline_data):
        """
        TODO: Initialize the complete MLOps pipeline.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Store all input data and model
        2. Initialize all MLOps components:
           - ModelMonitor with baseline accuracy
           - DriftDetector with baseline data
           - RetrainingTrigger with model and data
        3. Set up pipeline configuration:
           - monitoring_interval: 3600 (1 hour)
           - auto_retrain: True
           - deploy_threshold: 0.02 (2% improvement required)
        4. Initialize pipeline state:
           - pipeline_active: False
           - last_check_time: datetime.now()
           - deployment_history: []
        
        EXAMPLE USAGE:
        ```python
        pipeline = MLOpsPipeline(model, train_data, val_data, baseline_data)
        pipeline.start_monitoring()
        status = pipeline.check_system_health()
        ```
        
        IMPLEMENTATION HINTS:
        - Calculate baseline_accuracy from validation data (use 0.9 as default)
        - Use feature_names from data shape
        - Set reasonable defaults for all parameters
        - Initialize all components in __init__
        """
        ### BEGIN SOLUTION
    # YOUR CODE HERE
    raise NotImplementedError()
        ### END SOLUTION
    
    def start_monitoring(self):
        """
        TODO: Start the MLOps monitoring pipeline.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Set pipeline_active = True
        2. Update last_check_time = datetime.now()
        3. Log pipeline start
        4. Return status dictionary
        
        EXAMPLE RETURN:
        ```python
        {
            "status": "started",
            "pipeline_active": True,
            "start_time": datetime(2024, 1, 1, 12, 0),
            "message": "MLOps pipeline started successfully"
        }
        ```
        """
        ### BEGIN SOLUTION
    # YOUR CODE HERE
    raise NotImplementedError()
        ### END SOLUTION
    
    def check_system_health(self, new_data: Optional[np.ndarray] = None, current_accuracy: Optional[float] = None) -> Dict[str, Any]:
        """
        TODO: Check complete system health and trigger actions if needed.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Check if pipeline is active, return early if not
        2. Record current performance in monitor (if provided)
        3. Check for drift (if new_data provided)
        4. Check trigger conditions
        5. Execute retraining if needed (and auto_retrain is True)
        6. Return comprehensive system status
        
        EXAMPLE RETURN:
        ```python
        {
            "pipeline_active": True,
            "current_accuracy": 0.87,
            "drift_detected": True,
            "retraining_triggered": True,
            "new_model_deployed": True,
            "system_healthy": True,
            "last_check": datetime(2024, 1, 1, 12, 0),
            "actions_taken": ["drift_detected", "retraining_executed", "model_deployed"]
        }
        ```
        
        IMPLEMENTATION HINTS:
        - Use default values if parameters not provided
        - Track all actions taken during health check
        - Update last_check_time
        - Return comprehensive status for debugging
        """
        ### BEGIN SOLUTION
    # YOUR CODE HERE
    raise NotImplementedError()
        ### END SOLUTION
    
    def get_pipeline_status(self) -> Dict[str, Any]:
        """
        TODO: Get comprehensive pipeline status and history.
        
        STEP-BY-STEP IMPLEMENTATION:
        1. Get status from all components:
           - Monitor alerts and trends
           - Drift detection history
           - Retraining history
           - Deployment history
        2. Calculate summary statistics:
           - Total deployments
           - Average accuracy improvement
           - Time since last check
        3. Return comprehensive status
        
        EXAMPLE RETURN:
        ```python
        {
            "pipeline_active": True,
            "total_deployments": 3,
            "average_improvement": 0.05,
            "time_since_last_check": 300,
            "recent_alerts": [...],
            "drift_history": [...],
            "deployment_history": [...]
        }
        ```
        """
        ### BEGIN SOLUTION
    # YOUR CODE HERE
    raise NotImplementedError()
        ### END SOLUTION

### 🧪 Test Your Complete MLOps Pipeline

Once you implement the `MLOpsPipeline` class above, run this cell to test it:

In [ ]:
def test_mlops_pipeline():
    """Test complete MLOps pipeline"""
    print("🔬 Unit Test: Complete MLOps Pipeline...")
    
    # Create test data
    model = "test_model"
    train_data = np.random.normal(0, 1, (1000, 5))
    val_data = np.random.normal(0, 1, (200, 5))
    baseline_data = np.random.normal(0, 1, (1000, 5))
    
    # Create pipeline
    pipeline = MLOpsPipeline(model, train_data, val_data, baseline_data)
    
    # Test initialization
    assert pipeline.model == model
    assert pipeline.pipeline_active == False
    assert hasattr(pipeline, 'monitor')
    assert hasattr(pipeline, 'drift_detector')
    assert hasattr(pipeline, 'retrain_trigger')
    
    # Test start monitoring
    start_result = pipeline.start_monitoring()
    assert start_result["status"] == "started"
    assert start_result["pipeline_active"] == True
    assert pipeline.pipeline_active == True
    
    # Test system health check (no issues)
    health = pipeline.check_system_health(
        new_data=np.random.normal(0, 1, (100, 5)),
        current_accuracy=0.92
    )
    assert health["pipeline_active"] == True
    assert health["current_accuracy"] == 0.92
    assert "actions_taken" in health
    
    # Test system health check (with issues)
    health = pipeline.check_system_health(
        new_data=np.random.normal(5, 2, (100, 5)),  # Heavily drifted data
        current_accuracy=0.75  # Very low accuracy (well below 0.81 threshold)
    )
    assert health["pipeline_active"] == True
    assert health["drift_detected"] == True
    # Note: retraining_triggered depends on both accuracy and drift conditions
    # For fast testing, we just verify the system detects issues
    assert "retraining_triggered" in health
    
    # Test pipeline status
    status = pipeline.get_pipeline_status()
    assert status["pipeline_active"] == True
    assert "total_deployments" in status
    assert "average_improvement" in status
    assert "time_since_last_check" in status
    assert "recent_alerts" in status
    assert "performance_trend" in status
    
    print("✅ MLOpsPipeline initialization works correctly")
    print("✅ Pipeline start/stop functionality works")
    print("✅ System health checking works")
    print("✅ Drift detection and retraining integration works")
    print("✅ Pipeline status reporting works")
    print("📈 Progress: Complete MLOps Pipeline ✓")

# Run the test
test_mlops_pipeline()

## 🎯 Final Integration: Complete TinyTorch Ecosystem

### The Full System in Action
Let's demonstrate how all TinyTorch components work together in a complete MLOps pipeline:

```python
# Complete TinyTorch MLOps workflow
from tinytorch.core.tensor import Tensor
from tinytorch.core.networks import Sequential
from tinytorch.core.layers import Dense  
from tinytorch.core.activations import ReLU, Softmax
from tinytorch.core.training import Trainer, CrossEntropyLoss
from tinytorch.core.compression import quantize_layer_weights
from tinytorch.core.benchmarking import TinyTorchPerf
from tinytorch.core.mlops import MLOpsPipeline

# 1. Build model (Modules 01-04)
model = Sequential([
    Dense(784, 128), ReLU(),
    Dense(128, 64), ReLU(), 
    Dense(64, 10), Softmax()
])

# 2. Train model (Module 09)
trainer = Trainer(model, CrossEntropyLoss(), learning_rate=0.001)
trained_model = trainer.train(training_data, epochs=10)

# 3. Compress model (Module 10)
compressed_model = quantize_layer_weights(trained_model)

# 4. Benchmark model (Module 12)
perf = TinyTorchPerf()
benchmark_results = perf.benchmark(compressed_model, test_data)

# 5. Deploy with MLOps (Module 13)
pipeline = MLOpsPipeline(compressed_model, training_data, validation_data, baseline_data)
pipeline.start_monitoring()

# 6. Monitor and maintain
health = pipeline.check_system_health(new_data, current_accuracy=0.89)
if health["new_model_deployed"]:
    print("🚀 New model deployed automatically!")
```

### What Students Have Achieved
By completing this module, you have:
- **Built a complete ML system** from tensors to production deployment
- **Integrated all TinyTorch components** into a cohesive workflow
- **Implemented production-grade MLOps** with monitoring and automation
- **Created self-maintaining systems** that adapt to changing conditions
- **Mastered the full ML lifecycle** from development to production

### Real-World Impact
Your MLOps skills now enable:
- **Automated model maintenance** reducing manual intervention by 90%
- **Faster response to issues** from days to hours or minutes
- **Improved model reliability** through continuous monitoring
- **Scalable ML operations** that work across multiple models
- **Production-ready deployment** with industry-standard practices

In [ ]:
def test_comprehensive_integration():
    """Test complete integration of all TinyTorch components"""
    print("🔬 Comprehensive Integration Test: Complete TinyTorch Ecosystem...")
    
    # 1. Create synthetic data (simulating real ML dataset)
    np.random.seed(42)
    train_data = np.random.normal(0, 1, (1000, 10))
    val_data = np.random.normal(0, 1, (200, 10))
    baseline_data = np.random.normal(0, 1, (1000, 10))
    
    # 2. Create model architecture
    model = "TinyTorch_Production_Model"
    
    # 3. Set up complete MLOps pipeline
    pipeline = MLOpsPipeline(model, train_data, val_data, baseline_data)
    
    # 4. Start monitoring
    start_result = pipeline.start_monitoring()
    assert start_result["status"] == "started"
    print("✅ MLOps pipeline started successfully")
    
    # 5. Simulate production monitoring cycle
    print("\n🔄 Simulating Production Monitoring Cycle...")
    
    # Phase 1: Normal operation
    health1 = pipeline.check_system_health(
        new_data=np.random.normal(0, 1, (100, 10)),
        current_accuracy=0.94
    )
    print(f"   Phase 1 - Normal: Accuracy {health1['current_accuracy']}, Drift: {health1['drift_detected']}")
    
    # Phase 2: Gradual degradation
    health2 = pipeline.check_system_health(
        new_data=np.random.normal(0.5, 1, (100, 10)),
        current_accuracy=0.88
    )
    print(f"   Phase 2 - Degradation: Accuracy {health2['current_accuracy']}, Drift: {health2['drift_detected']}")
    
    # Phase 3: Significant drift and low accuracy
    health3 = pipeline.check_system_health(
        new_data=np.random.normal(2, 1, (100, 10)),
        current_accuracy=0.79
    )
    print(f"   Phase 3 - Critical: Accuracy {health3['current_accuracy']}, Drift: {health3['drift_detected']}")
    print(f"   Retraining triggered: {health3['retraining_triggered']}")
    print(f"   New model deployed: {health3['new_model_deployed']}")
    
    # 6. Get final pipeline status
    final_status = pipeline.get_pipeline_status()
    print(f"\n📊 Final Pipeline Status:")
    print(f"   Total deployments: {final_status['total_deployments']}")
    print(f"   Average improvement: {final_status['average_improvement']:.3f}")
    print(f"   System health: {health3['system_healthy']}")
    
    # 7. Verify complete integration
    assert final_status["pipeline_active"] == True
    assert len(final_status["deployment_history"]) >= 0
    assert "drift_history" in final_status
    assert "retrain_history" in final_status
    
    print("\n✅ Complete TinyTorch ecosystem integration successful!")
    print("🎉 All components working together seamlessly!")
    print("📈 Progress: Complete TinyTorch Ecosystem ✓")

# Run the comprehensive test
test_comprehensive_integration()

## 🧪 Auto-Discovery Testing

The following cell automatically discovers and runs all test functions in this module:

In [ ]:
if __name__ == "__main__":
    from tito.tools.testing import run_module_tests_auto
    
    # Automatically discover and run all tests in this module
    success = run_module_tests_auto("MLOps")

## 🎯 Module Summary: MLOps Production Systems

Congratulations! You've successfully implemented a complete MLOps system for production ML lifecycle management:

### What You've Built
✅ **Model Monitor**: Performance tracking and drift detection
✅ **Retraining Triggers**: Automated model updates based on performance thresholds
✅ **MLOps Pipeline**: Complete production deployment and maintenance system
✅ **Integration**: Orchestrates all TinyTorch components in production workflows

### Key Concepts You've Learned
- **Production ML systems** require continuous monitoring and maintenance
- **Drift detection** identifies when models need retraining
- **Automated workflows** respond to system degradation without manual intervention
- **MLOps pipelines** integrate monitoring, training, and deployment
- **System orchestration** coordinates complex ML component interactions

### Real-World Applications
- **Production AI**: Automated model maintenance at scale
- **Enterprise ML**: Continuous monitoring and improvement systems
- **Cloud deployment**: Industry-standard MLOps practices
- **Model lifecycle**: Complete deployment and maintenance workflows

### Connection to Industry Systems
Your implementation mirrors production platforms:
- **MLflow**: Model lifecycle management and experiment tracking
- **Kubeflow**: Kubernetes-based ML workflows and pipelines
- **Amazon SageMaker**: End-to-end ML platform with monitoring
- **Google AI Platform**: Production ML services with automation

### Next Steps
1. **Export your code**: `tito export 13_mlops`
2. **Test your implementation**: `tito test 13_mlops`
3. **Deploy production systems**: Apply MLOps patterns to real-world ML projects
4. **Complete TinyTorch**: You've mastered the full ML systems pipeline!

**🎉 TinyTorch Journey Complete!** You've built a complete ML framework from tensors to production deployment. You're now ready to tackle real-world ML systems challenges!